In [25]:
#### Librerias

import os
import giddy
import numpy as np
import pandas as pd
import seaborn as sns
import geopandas as gpd
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib.colors import LinearSegmentedColormap


import folium
from folium.plugins import TimestampedGeoJson



import warnings
warnings.filterwarnings("ignore", category = DeprecationWarning)

In [3]:
Mapa = gpd.read_file("MAP_UAD.json")
Time = pd.read_pickle("TIME_UNI.pkl")

In [12]:
Mapa.head(1)

,UNIDAD_ADM,Unidad,Local,geometry
0,R6_11A,GUERRERO 1,ACAPULCO,"POLYGON ((-99.494 18.721, -99.536 18.694, -99...."


In [11]:
New_Base = Time.melt(id_vars = ['CLAVE_UNIDAD'], 
                     var_name = 'time', 
                     value_name = 'valor')
New_Base['time'] = pd.to_datetime(New_Base['time'])
New_Base.head(1)

,CLAVE_UNIDAD,time,valor
0,R6_10B,2023-12-01,3.57348


In [16]:
Data_Markov = Mapa.merge(New_Base, 
                         left_on = 'UNIDAD_ADM',
                         right_on = 'CLAVE_UNIDAD', 
                         how = 'left').drop({'UNIDAD_ADM',
                                             'Unidad',
                                             'Local'}, axis = 1)
Data_Markov.head(3)

,geometry,CLAVE_UNIDAD,time,valor
0,"POLYGON ((-99.494 18.721, -99.536 18.694, -99....",R6_11A,2023-12-01,3.647723
1,"POLYGON ((-99.494 18.721, -99.536 18.694, -99....",R6_11A,2024-01-01,3.644675
2,"POLYGON ((-99.494 18.721, -99.536 18.694, -99....",R6_11A,2024-03-01,3.651950


In [24]:
# Paso 1: Crear una función para convertir valores a colores (usando Viridis, puedes cambiarlo)
def asignar_color(valor, vmin, vmax):
    norm = colors.Normalize(vmin=vmin, vmax=vmax)
    cmap = cm.get_cmap('YlOrRd')  # En esta seccion usar la cromatica creada para el mapa (la institucional)
    color = cmap(norm(valor))
    return colors.to_hex(color)

# Paso 2: Obtener los valores mínimo y máximo para escalar
vmin = Data_Markov['valor'].min()
vmax = Data_Markov['valor'].max()

# Paso 3: Agregar columna de color al GeoDataFrame
Data_Markov['color'] = Data_Markov['valor'].apply(lambda x: asignar_color(x, vmin, vmax))

# Paso 4: Convertir a GeoJSON con estilo personalizado
# Convertir columna 'time' a tipo datetime si no lo es
Data_Markov['time'] = pd.to_datetime(Data_Markov['time'])

# Ahora sí puedes aplicar .dt.strftime
Data_Markov['time'] = Data_Markov['time'].dt.strftime('%Y-%m-%d')

features = []
for _, row in Data_Markov.iterrows():
    features.append({
        'type': 'Feature',
        'geometry': row['geometry'].__geo_interface__,
        'properties': {
            'time': row['time'],
            'popup': f"{row['CLAVE_UNIDAD']}<br>Valor: {row['valor']:.2f}",
            'style': {
                'color': 'black',
                'weight': 0.5,
                'fillColor': row['color'],
                'fillOpacity': 0.7
            }
        }
    })

geojson_data = {
    'type': 'FeatureCollection',
    'features': features
}

# Paso 5: Crear el mapa
m = folium.Map(location=[19.3, -99.1], zoom_start=6)

# Paso 6: Agregar la animación
TimestampedGeoJson(
    geojson_data,
    period='P1M',
    add_last_point=False,
    transition_time=1000,
    auto_play=True,
    loop=False,
    date_options='YYYY-MM-DD',
    time_slider_drag_update=True
).add_to(m)

# Paso 7: Guardar
m.save("mapa_animado_coloreado.html")
